In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session
import tensorflow.keras.layers as L
import numpy as np
import pandas as pd
import sklearn
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


## Data preprocessing

In [2]:
train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
sample = pd.read_csv("/kaggle/input/digit-recognizer/sample_submission.csv")

In [3]:
clear_session()

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

x_train = train[train.columns[1:]]
y_train = train.label

x_train_float = x_train.astype(np.float) / 255 - 0.5 # нормализация данных
x_test_float = test.astype(np.float) / 255 - 0.5

y_train_oh = keras.utils.to_categorical(y_train, 10) # преобразование y_train в бинарную матрицу классов

x_train_float = x_train_float.values.reshape(-1, 28, 28) # подводка данных к нужной размерности
x_test_float = x_test_float.values.reshape(-1, 28, 28)


x_train_float,x_train_float_val,y_train_oh, y_train_oh_val = train_test_split(x_train_float, y_train_oh, random_state = 49, test_size = 0.3) 

## Deep learning

In [5]:
# Архитектура сети
model = Sequential()
model.add(L.Conv2D(16, kernel_size=3, strides=1, padding='same', input_shape=(28, 28, 1)))
model.add(L.MaxPool2D())
model.add(L.Conv2D(32, kernel_size=3, strides=1, padding='same'))
model.add(L.MaxPool2D())
model.add(L.Conv2D(64, kernel_size=3, strides=1, padding='same'))
model.add(L.MaxPool2D())
model.add(L.Flatten())
model.add(Dense(64, activation='relu', input_shape=(x_train.shape[1],))) # первый скрытый слой
model.add(Dense(32, activation='relu',)) # второй скрытый слой
model.add(Dense(10, activation='softmax',)) # выходной слой
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 576)               0

In [6]:
model.compile(
    loss='categorical_crossentropy',  # минимизируем кросс-энтропию
    optimizer='adam',  # так будет быстрее
    metrics=['accuracy']  # выводим процент правильных ответов
    
)

In [7]:
history = model.fit(
                    x_train_float[:, :, :, np.newaxis], 
                    y_train_oh,
                    batch_size=32,  # 32 объекта для подсчета градиента на каждом шаге
                    epochs=90,  # 90 проходов по датасету
                    validation_data = (x_train_float_val[:, :, :, np.newaxis], y_train_oh_val)
                    )

Epoch 1/90
919/919 [==============================] - 3s 4ms/step - loss: 0.2654 - accuracy: 0.9161 - val_loss: 0.0952 - val_accuracy: 0.9702
Epoch 2/90
919/919 [==============================] - 3s 4ms/step - loss: 0.0727 - accuracy: 0.9775 - val_loss: 0.0673 - val_accuracy: 0.9790
Epoch 3/90
919/919 [==============================] - 4s 4ms/step - loss: 0.0497 - accuracy: 0.9844 - val_loss: 0.0539 - val_accuracy: 0.9840
Epoch 4/90
919/919 [==============================] - 3s 4ms/step - loss: 0.0384 - accuracy: 0.9871 - val_loss: 0.0502 - val_accuracy: 0.9859
Epoch 5/90
919/919 [==============================] - 3s 4ms/step - loss: 0.0330 - accuracy: 0.9893 - val_loss: 0.0574 - val_accuracy: 0.9826
Epoch 6/90
919/919 [==============================] - 4s 4ms/step - loss: 0.0248 - accuracy: 0.9916 - val_loss: 0.0583 - val_accuracy: 0.9832
Epoch 7/90
919/919 [==============================] - 3s 4ms/step - loss: 0.0220 - accuracy: 0.9929 - val_loss: 0.0479 - val_accuracy: 0.9870
Epoch 

## Output

In [8]:
predictions = model.predict_classes(x_test_float[:, :, :, np.newaxis])

output = pd.DataFrame({'ImageId': sample.ImageId, 'Label': predictions})
output.to_csv('my_submission.csv', index=False)